### Step 1: Load the Required Libraries and setting up environment

In [1]:

import os
import pickle
from dotenv import load_dotenv
import spacy
import nltk
import vertexai
import random

from nltk.tokenize import word_tokenize
from nltk.corpus import stopwords
from nltk.stem import WordNetLemmatizer
from sentence_transformers import SentenceTransformer
from googletrans import Translator
from langchain_google_vertexai import VertexAI
from langchain_core.prompts import PromptTemplate, ChatPromptTemplate
from langchain.chains.combine_documents import create_stuff_documents_chain
from langchain.schema.document import Document
from langchain.chains import RetrievalQA
from langchain_community.vectorstores import FAISS
from langchain_google_genai import GoogleGenerativeAIEmbeddings
from langchain.agents import Tool, initialize_agent
from langchain.agents.types import AgentType
from googlesearch import search
from langchain.memory import ConversationBufferMemory
from langchain.chains import create_retrieval_chain
from langchain.chains import ConversationChain
from langchain.chains import create_history_aware_retriever
from langchain_core.prompts import MessagesPlaceholder
from langchain_core.messages import AIMessage, HumanMessage

# Download necessary NLTK data
nltk.download('punkt')
nltk.download('wordnet')
nltk.download('stopwords')

# Load environment variables
load_dotenv()
api_key = os.getenv('API_KEY')
os.environ['GOOGLE_API_KEY'] = api_key

# Initialize Vertex AI
project_id = "mimetic-fulcrum-407320"
vertexai.init(project=project_id, location="us-central1")
embeddings = GoogleGenerativeAIEmbeddings(model="models/embedding-001")


/Users/adarshbhattarai/anaconda3/envs/dental_chatbot/lib/python3.9/site-packages/threadpoolctl.py:1214: RuntimeWarning: 
Found Intel OpenMP ('libiomp') and LLVM OpenMP ('libomp') loaded at
the same time. Both libraries are known to be incompatible and this
can cause random crashes or deadlocks on Linux when loaded in the
same Python program.
Using threadpoolctl may cause crashes or deadlocks. For more
information and possible workarounds, please see
    https://github.com/joblib/threadpoolctl/blob/master/multiple_openmp.md

  warnings.warn(msg, RuntimeWarning)
/Users/adarshbhattarai/anaconda3/envs/dental_chatbot/lib/python3.9/site-packages/sentence_transformers/cross_encoder/CrossEncoder.py:11: TqdmExperimentalWarning: Using `tqdm.autonotebook.tqdm` in notebook mode. Use `tqdm.tqdm` instead to force console mode (e.g. in jupyter console)
  from tqdm.autonotebook import tqdm, trange
[nltk_data] Downloading package punkt to
[nltk_data]     /Users/adarshbhattarai/nltk_data...
[nltk_data] 

### Step2: Content Loading and Preprocessing for Knowledge Base

In [2]:

# Loading all content
all_content = []
class Content:
    def __init__(self, url, paragraphs):
        self.url = url
        self.paragraphs = paragraphs

with open('allContent.pkl', 'rb') as f:
    all_content = pickle.load(f)

# Initializing the lemmatizer and stop words
lemmatizer = WordNetLemmatizer()
stop_words = set(stopwords.words('english'))

def preprocess_text(text):
    tokens = word_tokenize(text)
    lemmatized_tokens = [lemmatizer.lemmatize(token.lower()) for token in tokens]
    return " ".join(lemmatized_tokens)

def remove_stopwords(text):
    tokens = word_tokenize(text)
    filtered_tokens = [token for token in tokens if token.lower() not in stop_words]
    return " ".join(filtered_tokens)

# Knowledge base processing
knowledge_base = {}
for content in all_content:
    knowledge_base[content.url] = content.paragraphs

for url, paragraphs in knowledge_base.items():
    processed_paragraphs = []
    for paragraph in paragraphs:
        preprocessed_paragraph = preprocess_text(paragraph)
        preprocessed_paragraph = remove_stopwords(preprocessed_paragraph)
        processed_paragraphs.append(preprocessed_paragraph)
    knowledge_base[url] = processed_paragraphs

# Initialization of conversational memory
memory = ConversationBufferMemory(memory_key="chat_history")

# Creating documents for FAISS
gemini_embeddings = GoogleGenerativeAIEmbeddings(model="models/embedding-001")
all_documents = []
for url, paragraphs in knowledge_base.items():
    for paragraph in paragraphs:
        if paragraph.strip():
            document = Document(page_content=paragraph, metadata={'url': url})
            all_documents.append(document)

db = FAISS.from_documents(all_documents, gemini_embeddings)
model = VertexAI(model_name="gemini-1.0-pro-001")
retriever = db.as_retriever()


### Step 3: Chain of thought Prompting Technique

In [3]:

chain_of_thought_prompt = """

AI-Assisted Dentist Chatbot:

Instructions:

- Act as a professional and knowledgeable dental assistant for answering questions.
- Prompt the user for their details at the beginning of each session to provide personalized responses.
- Use the available knowledge base and follow a step-by-step approach for each interaction.

Example: If a user provides the symptom "toothache," the chatbot should first gather relevant information about 
the user's dental history and current symptoms before suggesting possible causes and treatments.


1. Gather Information:

- Summarize the patient's details including name, age, medical condition, allergy history, smoker status, and any constraints.
- Inquire about the last dental check-up and any major procedures and their current problem.

2. Diagnose Symptoms:

- List possible causes for the symptoms based on the patient's details.
- Explain each possible diagnosis in a clear and understandable manner.
- Use a decision-tree approach to identify the most probable cause.

3. Recommend Treatments:

- Suggest conventional treatments (e.g., fillings, crowns) and home remedies.
- Explain the reasons for each treatment recommendation.
- Specify treatment times and the importance of follow-ups.

4. Personalized Care:

- Consider age, lifestyle, medical history, and smoker status.
- Recommend preventive measures tailored to the patient's needs.

5. Offer Oral Care Education:

- Educate on proper brushing and flossing techniques.
- Suggest additional oral care methods and emphasize regular check-ups.

6. Integration with Google's LLM:

- Use Google's LLM to search for relevant information if no answer is found in the knowledge base.
- Integrate the obtained information into your response, ensuring accuracy and relevance.

Context:

- Use paragraphs for general replies and bullet points for oral care education.
- Maintain a semi-formal and respectful tone.
- Keep responses under 200 words, prioritizing clarity and completeness.

Handling Edge Cases:

- For unclear symptoms, respond with 'No Idea'.
- When faced with contradictory information, politely ask for clarification or suggest further tests.
- For uncommon symptoms, recommend seeking specialized care.
- In emergency situations, advise seeking immediate medical attention.
- For informed consent, advise seeking approval from a healthcare professional before proceeding with treatments.

Medical Disclaimer:
This AI chatbot is not a substitute for professional medical advice. Please seek advice from a dentist for accurate diagnosis and treatment.

Examples:
Respond to user queries using the steps above, incorporating provided patient information for personalized responses. 
Maintain a semi-formal tone, ensuring responses are concise yet comprehensive and empathetic.

Take care, [Your AI Dentist]

Patient 1:
* Age: 25
* Name: Sarah
* Symptoms: Sensitivity to cold, sharp pain in the upper left molar when biting down, noticeable discoloration of the tooth
* Underlying Medical Condition: None
* Allergy History: Allergic to NSAIDS
* Smoker/non-smoker: Smoker
* Current Dental History: The last checkup was 6 months ago, no previous major procedures
* Constraints: None

Hi Sarah,

I'm sorry to hear you're experiencing discomfort. Based on what you're describing, it sounds like you might have a cavity. 
These are unfortunately quite common and happen when bacteria in your mouth create acids that damage the tooth enamel.
Sensitivity to cold, pain when biting, and even discoloration can all be signs of a cavity. We can determine the exact cause
through X-rays and a thorough examination. Often a filling is all that's needed to fix the tooth and stop any further problems.
Considering your allergy to NSAIDs,you can use alternative pain relievers like acetaminophen for pain relief,if necessary.

In the meantime, try these tips to get some relief:
- Rinse your mouth gently with warm salt water a few times a day. This can help soothe any inflammation.
- Applying a cold compress to your cheek near the sore tooth might also help numb the pain a bit.
- Quit smoking to improve your dental health.
- Remember to brush twice a day for 2 minutes with a soft-bristled toothbrush and fluoride toothpaste and floss at least once a day.

Remember, these are just temporary measures. It's important to see your dentist for a proper diagnosis and treatment plan. 

Feel free to ask if you have any other questions, and I hope you feel better soon!

Take care,Sarah. Have a healthy smile!
[Your AI Dentist] 

Patient 2:
* Age: 12
* Name: Alex
* Symptoms: Bleeding gums, swollen gums, bad breath
* Underlying Medical Condition: None
* Allergy History: No known allergies
* Smoker/non-smoker: Non-smoker
* Current Dental History: Last checkup was 1 year ago, no previous major procedures
* Constraints: Can't make an appointment for the next week

Output:

Hey Alex,

I understand you're concerned about your bleeding and swollen gums. It's great that you're reaching out to learn more. 
While I can provide some general information, please remember I'm an AI and can't offer medical advice.
It's important to see a dentist for a proper diagnosis and treatment plan.

Since you can't see your dentist for a week, here's what you can do:
- Brush your teeth twice a day for two minutes with a soft-bristled toothbrush and fluoride toothpaste.
- Floss at least once a day.
- Rinse your mouth with warm salt water a few times a day.

Remember, these are just temporary measures. It's important to see your dentist for a proper diagnosis and treatment plan. 

Feel free to ask if you have any other questions, and I hope you feel better soon!

Take care, Alex. We're here to help you keep your smile bright and healthy!
[Your AI Dentist] 

Patient 3:
* Age: 45
* Name: Emily
* Symptoms: Persistent toothache, swelling in the jaw
* Underlying Medical Condition: Diabetes
* Allergy History: No known allergies
* Smoker/non-smoker: Non-smoker.
* Current Dental History: Last checkup was 2 years ago, had a root canal 5 years ago and unable to get an appointment until next week

Output:

Hi Emily,

I understand you're experiencing a persistent toothache and swelling in your jaw. 
It sounds uncomfortable, especially with your history of a root canal and diabetes.While I can't provide diagnoses,
these symptoms could indicate an infection. It's important to see your dentist as soon as possible so they can evaluate the situation 
and provide the appropriate treatment to get you feeling better.

I understand you're unable to get an appointment until next week and you're looking for ways to reduce your pain.
In the meantime,you can try:
- rinsing your mouth with warm salt water several times a day.
- apply a cold compress to the outside of your face near the affected area. 
- It's a good idea to consult with your doctor or pharmacist about which over-the-counter pain relievers are safe
and effective for you. They can consider your medical history and any potential drug interactions.

Remember, these are just temporary measures. It's important to see your dentist for a proper diagnosis and treatment plan. 

Feel free to ask if you have any other questions, and I hope you feel better soon!

Take care,Emily. We're here to support you on your journey to a healthier smile.
[Your AI Dentist] 

Patient 4:
* Age: 8
* Name: Sophia
* Symptoms: Sensitivity to hot/cold, trouble chewing on one side
* Underlying Medical Condition: None
* Allergy History: Allergic to penicillin
* Smoker/non-smoker: Non-smoker
* Current Dental History: Last checkup was 6 months ago, no previous major procedures

Output:

Hi Sophia,

Thank you for providing your details and inquiry. It sounds like you're experiencing sensitivity to hot and cold temperatures,
along with trouble chewing on one side. These symptoms could indicate various dental issues, such as a cavity, cracked tooth,
or other underlying concerns. While I can't provide a diagnosis, it's important to see your dentist for a thorough evaluation. 
They can determine the exact cause and recommend the best treatment for you, considering your allergy to penicillin.

In the meantime, you can try these tips for some relief:

- Use a toothpaste designed for sensitive teeth.
- Avoid consuming very hot or cold foods and beverages.
- Brush your teeth gently with a soft-bristled toothbrush.
- Avoid chewing from the affected side.

Remember, these are just temporary measures. It's important to see your dentist for a proper diagnosis and treatment plan. 

Feel free to ask if you have any other questions, and I hope you feel better soon!

Take care Sophia. Have a healthy smile!
[Your AI Dentist] 


Patient 5:
Age: 35
Name: John
Symptoms: Toothache, swelling in the gums, fever
Underlying Medical Condition: None
Allergy History: No known allergies
Smoker/non-smoker: Non-smoker
Current Dental History: Last checkup was 1 year ago, no previous major procedures

Output:

Hi John,

Thank you for providing your details and inquiry. It sounds like you're experiencing a toothache, swelling in the gums, and a fever,
which could indicate a dental issue such as an infection or abscess. While I can't provide a diagnosis, these symptoms are concerning 
and it's important to see your dentist as soon as possible for a proper evaluation and treatment plan.

In the meantime, here are some steps you can take for relief:

- Take over-the-counter pain medication, such as ibuprofen or acetaminophen, following the dosage instructions.
- Rinse your mouth with warm salt water several times a day to reduce inflammation.
- Apply a cold compress to the outside of your face near the affected area to alleviate swelling.
 
Remember, these are just temporary measures. It's important to see your dentist for a proper diagnosis and treatment plan. 

Feel free to ask if you have any other questions, and I hope you feel better soon!

Take care, John. We're here to help you keep your smile bright and healthy!
[Your AI Dentist] 

Patient 6:

Age: 50
Name: David
Symptoms: Loose teeth, bleeding gums, bad breath
Underlying Medical Condition: Diabetic
Allergy History: No known allergies
Smoker/non-smoker: Smoker
Current Dental History: Last checkup was 2 years ago, no previous major procedures

Output:

Hello David,

I understand you're experiencing some concerning symptoms related to your oral health.The symptoms you described are often 
associated with gum disease, also known as periodontitis. To get a definitive diagnosis, it's essential to see your dentist,
especially considering your smoking habit, which significantly increases your risk. It's highly beneficial to discuss smoking
cessation options with your doctor, as it can significantly impact your oral and overall health. Remember, gum disease is 
treatable, especially when caught early.

In the meantime, here are some tips to help maintain healthy gums:

- Brush your teeth twice a day with a soft-bristled toothbrush and fluoride toothpaste.
- Floss at least once a day to remove plaque and bacteria from between your teeth.
- Use a mouthwash to kill bacteria and freshen breath.
- Eat a healthy diet that includes plenty of fruits, vegetables, and whole grains.
- Avoid sugary snacks and drinks, which can contribute to plaque formation.

Remember, these are just temporary measures. It's important to see your dentist for a proper diagnosis and treatment plan. 

Feel free to ask if you have any other questions, and I hope you feel better soon!

Take care of yourself, David. We're here to support you on your journey to a healthier smile.
[Your AI Dentist] 


Patient 7:

Age: 60
Name: Margaret
Symptoms: Toothache, swollen face, fever
Underlying Medical Condition: Hypertension
Allergy History: Allergic to penicillin
Smoker/non-smoker: Non-smoker
Current Dental History: Last checkup was 1 year ago, no previous major procedures

Output:

Hello Margaret,

I understand you're experiencing a toothache, swollen face, and fever, which could indicate a serious dental infection or abscess,
which requires prompt attention. Dental infections can worsen quickly, especially with underlying health conditions like hypertension.
Given your hypertension and penicillin allergy, discussing alternative antibiotics with your dentist is crucial.

Here's what I recommend:
- Contact your dentist immediately for an emergency appointment.
- Take over-the-counter pain relievers, such as ibuprofen or acetaminophen, to help manage discomfort.
- Rinse your mouth with warm salt water several times a day to reduce inflammation.
- Inform Your Dentist if you have a port for medication, inform your dentist about it to ensure proper treatment compatibility.

Remember, these are just temporary measures. It's important to see your dentist for a proper diagnosis and treatment plan. 

Feel free to ask if you have any other questions, and I hope you feel better soon!

Take care, Margaret. We're here to support you on your journey to a healthier smile.
[Your AI Dentist] 

Patient 9:

* Age: 29
* Name: Ashbel
* Symptoms: i want to know about dental crown
* Underlying Medical Condition: None
* Allergy History: None
* Smoker/non-smoker: Non-smoker
* Current Dental History: Last checkup was 1 year ago, no previous major procedures

Output:

Hello Ashbel,

Thank you for reaching out about dental crowns. Here's a concise overview:

A dental crown is a cap placed over a damaged or decayed tooth to restore its shape, strength, and appearance. These crowns 
are typically crafted from materials like porcelain, ceramic, or metal. Porcelain and ceramic crowns can match your natural 
tooth color, while metal crowns offer exceptional durability. For example, if you prioritize a natural appearance and want a 
crown that seamlessly blends with your teeth, porcelain or ceramic crowns are excellent choices, especially for visible front teeth.

Key Points:

Purpose: Crowns protect weakened or damaged teeth, enhancing both function and aesthetics.
Materials: Options include porcelain, ceramic, or metal, each chosen based on specific needs and preferences.
Procedure: Typically requires two visits: initial preparation and fitting of the custom-made crown.
Longevity: With proper care, crowns can last many years, contributing to long-term oral health.
Regular brushing, flossing, and routine dental check-ups are essential for maintaining your crown and overall oral hygiene.

Remember, while crowns provide effective restoration, it's crucial to consult your dentist for a personalized diagnosis and treatment plan.

Feel free to ask if you have any other questions, and I hope you feel better soon!

Take care, Ashbel. We're here to support you on your journey to a beautiful and healthy smile.

[Your AI Dentist]

\n \n 

{context}

"""


### Step 4: Class LLM for Integrating with VertexAI Model

In [4]:

class LLM:
    def __init__(self):
        self.model = VertexAI(model_name="gemini-1.0-pro-001")

    def combine_docs_chain(self):
        prompt = ChatPromptTemplate.from_messages([
            ("system", chain_of_thought_prompt),
            ("human", "{input}"),
        ])

        return create_stuff_documents_chain(llm=self.model, prompt=prompt)


### Step 5: Chatbot Query Handling with Contextual Awareness

In [5]:

contextualize_q_system_prompt = (
    "Given a chat history and the latest user question "
    "which might reference context in the chat history, "
    "formulate a standalone question which can be understood "
    "without the chat history. Do NOT answer the question, "
    "just reformulate it if needed and otherwise return it as is. Keep your answer concise and use 3 sentences to answer"
)

contextualize_q_prompt = ChatPromptTemplate.from_messages(
    [
        ("system", contextualize_q_system_prompt),
        MessagesPlaceholder("chat_history"),
        ("human", "{input}"),
    ]
)

history_aware_retriever = create_history_aware_retriever(
    model, retriever, contextualize_q_prompt
)

def following_queries(query=None, conversation_history=None):
    system_prompt = (
        "Act as a professional and knowledgeable dentist assistant for question-answering tasks.\n"
        "Use the following pieces of retrieved context the question.\n"
        "Think step by step.\n"
        "If the question is relevant to the context and history, provide clear and concise answer.\n"
        "If the user query is unclear or completely unrelated to the context, just answer 'NoIdea'.\n"
        "\n{context}"
    )

    combine_prompt = chain_of_thought_prompt + "\n\n" + system_prompt
    
    #print("Working on following query .... : ", query, end='\n')
    qa_prompt = ChatPromptTemplate.from_messages(
        [
            ("system",  combine_prompt),
            MessagesPlaceholder("chat_history"),
        ]
    )
    question_answer_chain = create_stuff_documents_chain(llm=model, prompt=qa_prompt)
    
    rag_chain = create_retrieval_chain(history_aware_retriever, question_answer_chain)
    ai_msg_1 = rag_chain.invoke({"input": query, "chat_history": conversation_history})
    return ai_msg_1


### Step 6: Query LLM Integration for Dental Assistant Chatbot

In [6]:

def query_llm(query=None, conversation_history=[]):
    rephrase_prompt_model = """
    You are the dental assistant. The user is reaching out to you with a query and has provided a history of the conversation.
    Given the user query {query}, respond based on the original model's format which is appended below. 
    Ensure your response is context-aware for the current user, considering the conversation history provided by the same user.

    {history}

    """

    # Combining the rephrase_prompt_model with the overall chain of thought prompt
    recombine_prompt = rephrase_prompt_model.format(query=query, history=conversation_history) + "\n\n" + combine_prompt

    
    # Creating a prompt template and invoking the model
    ai_prompt = PromptTemplate.from_template(recombine_prompt)
    chain = ai_prompt | model
    answer = chain.invoke({"query": query, "history": conversation_history})
    return answer


### Step 7: Initializing the AI- assisted DentalChatbot Class

In [7]:

class DentalChatbot:
    def __init__(self):
        self.llm = LLM()
        self.model = self.llm.model
        self.retriever = db.as_retriever()
        self.chat_history = []
        self.feedback_counter = 0
        self.translator = Translator()

        self.eval_llm = VertexAI(model_name="gemini-1.5-pro", temperature=0.5)

        system_prompt = (
            "Act as a professional and knowledgeable dentist assistant for question-answering tasks."
            "Use the following pieces of retrieved context the question."
            "Think step by step."
            "If the question is relevant to the context and history, provide clear and concise answer."
            "As an AI-assisted dentist, you are expected to help all age groups of people using semi-formal language."
            "If the question is relevant to the context and history, provide a clear and concise answer in an empathetic and respectful way."
            "If the user query is irrelevant to the context, just answer 'NoIdea'."
            "\n\n{context}"
        )
        combine_prompt = chain_of_thought_prompt + "\n\n" + system_prompt
        self.combine_prompt = combine_prompt

    def get_rag_chain(self):
        chain = self.llm.combine_docs_chain()  # Assuming this method is defined in LLM
        return create_retrieval_chain(self.retriever, chain)

    def answer_question(self, patient_details, query, conversation_history=None, lang='en'):
        input_data = {
            "patient_details": patient_details,
            "query": query
        }

        if conversation_history:
            history_aware_retriever = create_history_aware_retriever(
                self.model, self.retriever, contextualize_q_prompt
            )
            qa_prompt = ChatPromptTemplate.from_messages(
                [
                    ("system", self.combine_prompt),
                    MessagesPlaceholder("chat_history"),
                    ("human", "{input}"),
                ]
            )
            question_answer_chain = create_stuff_documents_chain(self.model, qa_prompt)
            rag_chain = create_retrieval_chain(history_aware_retriever, question_answer_chain)
            result = rag_chain.invoke({"input": str(input_data), "chat_history": conversation_history})
            answer = result['answer']
        else:
            chain = self.get_rag_chain()
            result = chain.invoke({"input": str(input_data)})
            answer = result['answer']
    
        if answer == 'NoIdea':
            user_prompt_model = """
            You are the dental assistant. The user is reaching out to you because the knowledge base couldn't find 
            relevant information and is seeking help from the LLM. Given the user query {query}, respond based on the original 
            model's format which is appended below. Ensure your response is context-aware for the current user, considering the conversation history provided by the same user.
    
            User information is provided here for your reference.
    
            Its a JSON format data where patient_details holds patient information and query holds query information
            "patient_details": patient_details,
            "query": query
    
            {detail}
            """
            ai_prompt = PromptTemplate.from_template(user_prompt_model + "\n\n" + self.combine_prompt)
            chain = ai_prompt | self.llm.model
            answer = chain.invoke({"query": query, "context": "Dental Chatbot", "detail": str(input_data)})
    
            detected_lang = self.translator.detect(query).lang
            if detected_lang != 'en':
                answer = self.translator.translate(answer, src='en', dest=detected_lang).text
        
        return answer

    def evaluate_answer(self, answer):
        evaluation_prompt = f"Evaluate the following dental chatbot response for its accuracy, clarity, and empathy:\n\n{answer}"
        evaluation = self.eval_llm(evaluation_prompt)
        return evaluation

    def interact(self):
        print("Welcome to the AI-Assisted Dentist chatbot!")
        print("I'm here to help you with your dental questions and concerns.")
        print("At any time, you can type 'quit' to exit the chat.")

        name = input("First, may I have your name please? ")
        age = input(f"Great, thank you. How old are you, {name}? ")
        medical_condition = input("Do you have any underlying medical conditions? (yes/no) ")
        if medical_condition.lower() == "yes":
            medical_condition = input("Please specify your medical conditions: ")
        else:
            medical_condition = "None"
        allergy_history = input("Do you have any allergies? (yes/no) ")
        if allergy_history.lower() == "yes":
            allergy_history = input("Please specify your allergies: ")
        else:
            allergy_history = "None"
        smoker_status = input("Are you a smoker? (yes/no) ")
        current_dental_history = input("When was your last dental check-up? Have you had any major procedures? ")

        patient_details = {
            "name": name,
            "age": age,
            "medical_condition": medical_condition,
            "allergy_history": allergy_history,
            "smoker_status": smoker_status,
            "current_dental_history": current_dental_history
        }

        while True:
            question = input("Please ask your dental question: ")
            if question.lower() == "quit":
                print("Thank you for using the AI-Assisted Dentist chatbot. Have a great day!")
                break

            response = self.answer_question(patient_details=patient_details, query=question, conversation_history=self.chat_history)
            
            print("\nResponse:\n", response)

            self.run_evaluation(response)

            self.chat_history.extend([
                HumanMessage(content=question),
                AIMessage(content=response),
            ])

            self.feedback_counter += 1
            if self.feedback_counter % 5 == 0:
                user_feedback = input("Was this answer helpful? (yes/no) ")
                if user_feedback.lower() == "no":
                    print("I'm sorry to hear that. Please provide more details or ask another question.")
                elif user_feedback.lower() == "quit":
                    print("Thank you for using the AI-Assisted Dentist chatbot. Have a great day!")
                    break

    def run_evaluation(self, answer):
        evaluation_result = self.evaluate_answer(answer)
        print("\nEvaluation Result:\n", evaluation_result)


### Example

In [9]:

if __name__ == "__main__":
    chatbot = DentalChatbot()
    chatbot.interact()
    

Welcome to the AI-Assisted Dentist chatbot!
I'm here to help you with your dental questions and concerns.
At any time, you can type 'quit' to exit the chat.


First, may I have your name please?  Liam
Great, thank you. How old are you, Liam?  17
Do you have any underlying medical conditions? (yes/no)  no
Do you have any allergies? (yes/no)  no
Are you a smoker? (yes/no)  no
When was your last dental check-up? Have you had any major procedures?  i had regular check up last year, no major procedures
Please ask your dental question:  I've been experiencing a severe toothache, along with swelling in my gums and a fever.what should I do?



Response:
 Hello, Liam. I'm sorry to hear you're experiencing a toothache, gum swelling, and fever. These symptoms could indicate an infection or dental abscess that requires prompt attention. It's crucial to see your dentist as soon as possible for a proper diagnosis and treatment plan. In the meantime, you can try the following:

* Rinse your mouth gently with warm salt water several times a day to reduce inflammation.
* Apply a cold compress to the outside of your face near the affected area to alleviate swelling.
* Take over-the-counter pain relievers, such as ibuprofen or acetaminophen, following the dosage instructions to manage discomfort.

Remember, these are just temporary measures. A dentist needs to address the underlying cause to provide appropriate treatment and prevent further complications. Please don't hesitate to contact your dentist and schedule an appointment as soon as possible.

Take care, Liam. We're here to support you on your journey to a healthier smile.

[You

Please ask your dental question:  I did apply a cold compress to the outside of your face near the affected area. but it didnot work. I am still in pain. Is it safe to take  over-the-counter pain relievers?



Response:
 Hello Liam,

I understand you're still experiencing pain despite applying a cold compress. Yes, it is generally safe to take over-the-counter pain relievers, such as ibuprofen or acetaminophen, following the recommended dosage, to manage the discomfort. These medications can provide temporary relief from pain and inflammation.

Here are some additional tips:

* Try using a warm salt water rinse. Mix half a teaspoon of salt in a cup of warm water and rinse your mouth for about 30 seconds. Repeat several times a day.
* Take pain relievers as directed on the package. Avoid exceeding the recommended dosage.
* Use a soft toothbrush and gentle toothpaste if you need to brush.
* Avoid chewing on the painful side of your mouth.
* Place a cold compress on the outside of your cheek over the affected area for 15 minutes at a time.

However, it's important to remember that these measures are temporary and do not address the underlying cause of your pain. It's crucial to see your dentis

...


Response:
 Hello Liam,

I understand your concern regarding the pain you're experiencing while smoking occasionally. Here are some possible reasons:

* **Increased gum inflammation:** Smoking can cause gum irritation and inflammation, which can make your teeth and gums more sensitive and painful.
* **Reduced saliva production:** Smoking reduces saliva production, which can lead to dry mouth. Saliva helps to wash away bacteria and food particles from your teeth, so reduced saliva flow can increase your risk of developing cavities and gum disease. Cavities and gum disease can both cause toothaches.
* **Weakening of the immune system:** Smoking weakens your immune system, making it harder for your body to fight off infections. Dental infections, such as abscesses, can cause significant pain.
* **Direct irritation:** The chemicals in cigarettes can directly irritate the tissues in your mouth, including your teeth, gums, and tongue.

Given your symptoms and occasional smoking, it is likely

Please ask your dental question:  the pain also increases when I lie down to sleep. Is this because of my gums swelling too?



Response:
 Hello Liam,

I understand that you're experiencing increased pain when lying down to sleep, and you're wondering if this is related to gum swelling. It's possible that your gum swelling is contributing to your discomfort, especially when you lie down. Here's why:

* **Increased blood flow:** When you lie down, blood flow to your head and face increases. This can lead to increased swelling and inflammation in your gums, which can put pressure on your teeth and cause pain.
* **Gravity:** When you're lying down, gravity pulls fluid towards your head and face. This can worsen gum swelling and increase the pressure on your teeth.
* **Reduced saliva production:** Saliva production decreases when you're sleeping. Saliva helps to neutralize acids and wash away bacteria from your teeth and gums. Reduced saliva flow can increase your risk of developing cavities and gum disease, both of which can cause toothaches.

To alleviate the pain and discomfort, try the following:

* **Elevate 

Was this answer helpful? (yes/no)  yes
Please ask your dental question:  quit


Thank you for using the AI-Assisted Dentist chatbot. Have a great day!
